# 并行计算

cython中当然可以正常使用多进程多线程和协程,但至少目前由于gil的限制这条技术路线是无法高效的利用多核的.Cython提供了对解放gil的支持,同时可以借助openmp提供的c级别多线程能力真正做到充分利用多核.

In [1]:
%load_ext Cython

## 突破GIL限制

我们都知道由于GIL的限制,python有个假的多线程--无论多少线程都只会使用1个cpu.这以现状可能会在后续的版本中有所突破,但至少目前不行.

并不是所有情况下都应该或者需要突破GIL限制,突破GIL限制的代价是代码复杂性和可维护性,事实上只有如下情况会真正受益于突破GIL限制:

+ 当你使用Cython的openmp进行并行计算时
+ 当你打算将没有副作用的重型任务交给python的多线程在后台处理时.这个重型任务可以是计算密集型也可以是io密集型,只要够重就划得来
+ 当你的Cython代码运行时间较长且从未调用Python解释器时,需要使用`with nogil: pass`来暂时的释放GIL以避免死锁

在`nogil`模式下你需要小心管理线程间的数据共享和同步以避免竞态条件和数据一致性问题.

### `nogil`模式的使用限制

GIL是Python的底层机制,几乎python对象的所有操作都会触发GIL,因此绕过GIL因此`nogil`模式要跳过gil的限制会有如下限制

1. 不能有python对象
2. 不能触发异常,异常也是python对象,因此必须保证调用的代码逻辑不会触发异常.


### 声明一个函数受不受GIL限制

我们可以直接声明一个纯处理C代码逻辑的C函数不受GIL限制,使用如下语法

```cython
cdef void some_func() noexcept nogil:
    pass
```

需要注意这个函数通常同样有`noexcept`标识,这是因为异常的触发是会触发GIL的校验的.

`nogil`模式的函数不能有任何Python参数或Python对象的返回值,内部如果需要与Python交互则需要使用上下文`with gil:`或调用明确声明为`with gil`的C函数.

注意nogil函数注释声明在没有GIL的情况下调用该函数是安全的.完全可以在持有GIL的同时执行它.如果调用者持有该功能,本身并不释放GIL.

In [2]:
%%cython
# distutils: extra_compile_args=-Wno-unreachable-code

cdef int add(int a, int b) noexcept nogil:
    return a + b

print(add(1,2))

3


类似的我们也可以声明一个C函数必定受GIL限制,使用如下语法:

```cython
cdef void some_func() [except 标质量] with gil:
    pass
```

这样的函数将无论怎样都受gil限制,因为它会在调用时主动获取gil在执行完成后释放gil.

通常`with gil`标识的函数和`nogil`标识的函数一样都是参数,返回值,内部都没有python对象和异常的C函数,正因为如此才需要标识为`with gil`主动触发,否则完全没有必要

### 声明一段代码受不受GIL限制

Cython提供了上下文管理器`gil`和`nogil`用于控制在他们`with`语句内的代码块是否受GIL限制.

```cython
with nogil:
    ...    # 代码块内不受GIL限制
    with gil:
        ...  # 代码块内受GIL限制
    ...   # 代码块内继续不受GIL限制
```

这两个上下文管理可以结合函数GIL限制的声明一起使用:

```cython
cdef void some_func() with gil: # 受GIL限制的函数
    ...
    
with nogil:
    ...    # 代码块内不受GIL限制
    some_func() # 代码块内受GIL限制
    ..   # 代码块内继续不受GIL限制
```

#### 条件GIL控制限制上下文

我们可以使用如下语法控制**编译时**是否使用GIL控制上下文,

```cython
with nogil(some_type is not object):
    ...
```

注意这是编译时行为,这主要是在使用模版时会用到.

### 异常与GIL

异常总是需要GIL,因此在`nogil`模式下Cython会隐式地重新获取它.这就是为什么在`with nogil:`上下文中我们必须保证操作不会触发异常.

类似地如果声明为nogil的函数抛出异常,`nogil`模式也会一定程度上失效.Cython虽然可以正确地传播它而无需编写显式代码来处理,但在触发异常时一样也会重新获取GIL.特别是在异常标志量为`except *`的函数中,由于这类异常无论如何触发都要调取CPython的ABI,因此异常触发效率较低且每次调用都会执行,因此这类函数在调用的时候`nogil`模式都会失效.

## 使用openmp并行计算

Cython提供了对openmp的原生支持,不仅有标准库而且提供了高级接口以便更方便的使用.

### 编译设置

要使用openmp需要有`libomp`,这个库在mac下默认的clang中是没有的,如果需要用建议直接安装gcc解决.总体来说gcc在哪个平台都挺靠谱,在mac上还可以避免和自带的clang冲突,就都建议作为默认编译器使用

1. 安装gcc

    ```bash
    brew install gcc
    ```

2. 设置环境变量(以gcc13.1.0为例)

    ```bash
    alias gcc=/usr/local/Cellar/gcc/13.1.0/bin/gcc-13
    alias g++=/usr/local/Cellar/gcc/13.1.0/bin/g++-13
    alias gfortran=/usr/local/Cellar/gcc/13.1.0/bin/gfortran-13
    export CC=/usr/local/Cellar/gcc/13.1.0/bin/gcc-13
    export CXX=/usr/local/Cellar/gcc/13.1.0/bin/g++-13
    ```
    
3. 在编译时带上编译器参数`extra_compile_args=['-fopenmp']`和`extra_link_args=['-fopenmp']`



### 结合Cython回顾openmp的用法

openmp使用编译器指令`#pragma omp xxx`和特定函数来控制程序并行执行.在Cython中并没有全部提供,下面是Cython用到的openmp编译器指令和函数.


#### Cython中的openmp编译器指令

Cython中的openmp编译器指令靠`prange`,`range`和`with parallel():`三种代码块来进行翻译,具体的用法我们会在后面的高级接口部分介绍,这3种代码块会被转译为的openmp编译器指令有如下:

+ `#pragma omp parallel`, 用于在一个代码块之前,标明这个代码块会被多个线程执行
+ `private(i)`,通常结合在`#pragma omp parallel`中,标明这个代码块中的变量`i`在每个线程中都有自己的私有变量
+ `reduction(+:total)`,通常结合在`#pragma omp parallel`中,标明这个代码块中的变量`total`为归约变量,它会执行规约运算`+`,Cython中支持的归约运算包括`+`,`*`,`-`,`&`, `|`,`^`,**并不支持`||`和`&&`**
+ `#pragma omp for ...`, 用于`for`循环语句前,标明将循环中的计算任务分配到多个线程中执行,注意必须保证每个循环中的数据不相关
+ `firstprivate(i)`,必须配合`#pragma omp for ...`使用,表明变量`i`的在每个线程中都有自己的私有变量,并且私有变量会在进入并行域或任务分配时继承主线程中变量的值作为初值
+ `lastprivate(i)`,必须配合`#pragma omp for ...`使用,表明变量`i`作为每个线程中的私有变量,在处理完后会被赋值到主线程中的同名变量中

#### Cython中的openmp函数

openmp中的函数一般是用来设置或获取线程相关信息的.其中比较有用的被收在Cython的标准库`openmp`中其中比较重要的包括

函数名|作用
---|---
`omp_in_parallel`|判断当前是否在并行域中
`omp_get_thread_num`|返回线程号
`omp_set_num_thread`|设置后续并行域中的线程格式
`omp_get_num_threads`|返回当前并行域中的线程数
`omp_get_max_threads`|返回并行域可用的最大线程数目
`omp_get_num_procs`|返回系统中处理器的数目
`omp_get_dynamic`|判断是否支持动态改变线程数目
`omp_set_dynamic`|启用或关闭线程数目的动态改变
`omp_get_nested`|判断系统是否支持并行嵌套
`omp_set_nested`|启用或关闭并行嵌套

除此之外还有`cython.parallel`库中的`threadid()`方法,它可以获取当前线程的id

In [3]:
%%cython
# distutils: extra_compile_args=-fopenmp
# distutils: extra_link_args=-fopenmp

from cython.parallel import parallel, threadid
cimport openmp

cdef int num_threads, num_procs, threadid

openmp.omp_set_dynamic(1)
with nogil, parallel():
    num_threads = openmp.omp_get_num_threads()
    num_procs = openmp.omp_get_num_procs()
    threadid = threadid()
    with gil:
        print(f"threadid: {threadid},num_threads: {num_threads},num_prpces: {num_procs}")

threadid: 0,num_threads: 5,num_prpces: 8threadid: 1,num_threads: 5,num_prpces: 8
threadid: 2,num_threads: 5,num_prpces: 8
threadid: 3,num_threads: 5,num_prpces: 8

threadid: 4,num_threads: 5,num_prpces: 8


### 高级接口

Cython通过python库`cython.parallel`模块支持本机并行计算.当不进行编译或没有openmp设置时,这些高级接口会退化为使用原生python的方式执行(和纯净模式中`cython`库的行为一致),但当你有上面的编译设置,且进行了编译,他们会被转译为特定的openmp编译器指令,从而提供基于openmp的多线程能力

cython中提供的高级接口有两个:

+ 上下文管理器`parallel`,用于声明其中的代码会并行执行
+ 类`range`函数`prange`,用于使用`for`循环语法调度分发任务


#### parallel

上下文管理器`parallel`用于声明其中的代码段交给多线程执行,其签名如下

`cython.parallel.parallel(num_threads=None)->ContextManager[None]`

`num_threads=None`时线程数量取决于系统设置,而设置为其他值时就设多少就起多少个线程.

`parallel`一般用的时候会和`nogil`上下文管理器一起使用,上面已经有例子了.

其在没有编译的情况下,它没有效果

In [4]:
import cython
from cython.parallel import parallel, threadid

threadid:cython.int

# openmp.omp_set_dynamic(1)
with cython.nogil, parallel(num_threads=4):
    threadid = threadid()
    with cython.gil:
        print(f"threadid: {threadid}")

threadid: 0


当进行编译时,它会转译为类似如下的openmp编译器指令

```C
#pragma omp parallel
{
    /* body goes here */
}
```

如果在其中有用到什么变量,则这些变量会被自动填入`private(i)`

比如上面例子就会转译为

```C
#pragma omp parallel private(threadid)
{
    /* body goes here */
}
```

In [5]:
%%cython
# distutils: extra_compile_args=-fopenmp
# distutils: extra_link_args=-fopenmp

from cython.parallel import parallel, threadid
cimport openmp

cdef int threadid
openmp.omp_set_dynamic(1)
with nogil, parallel(num_threads=4):
    threadid = threadid()
    with gil:
        print(f"threadid: {threadid}")

threadid: 1threadid: 0
threadid: 3

threadid: 2


#### prange

prange用于使用`for`语法控制并行任务.其接口为

`cython.parallel.prange([start,] stop[, step][, nogil=False][, schedule=None[, chunksize=None]][, num_threads=None])->Iterable[int]`

    
`prange`表现上类似python内置函数`range`.
    
一个

```cython
for i in prange(): 
    pass
```

语句被转译后等价于类似如下

```C
#pragma omp parallel
{
    #pragma omp for firstprivate(i) lastprivate(i)
    for (__pyx_t_8 = 0; __pyx_t_8 < __pyx_t_9; __pyx_t_8++){
        i = __pyx_t_8;
        /* body goes here */
    }
}
```

如果其中还包含`for j in range(xxx):`,那`j`也会被设置到`lastprivate`,类似

```C
#pragma omp parallel
{
    #pragma omp for firstprivate(i) lastprivate(i) lastprivate(j)
    for (__pyx_t_8 = 0; __pyx_t_8 < __pyx_t_9; __pyx_t_8++){
        i = __pyx_t_8;
        /* body goes here */
    }
}
```

如果其中还有`+=`, `*=`, `-=`, `&=`, `|=`,`^=`这6个运算符,则还会增加`reduction`指令,归约运算与`=`前的一致,归约变量为符号左侧的变量.比如

```cython
cdef total = 0
for i in prange(10): 
    total += i
```
就会被转译为
```C
#pragma omp parallel reduction(+:total)
{
    #pragma omp for firstprivate(i) lastprivate(i)
    for (__pyx_t_8 = 0; __pyx_t_8 < __pyx_t_9; __pyx_t_8++){
        i = __pyx_t_8;
        /* body goes here */
    }
}
```

另外需要注意,**在循环中我们无法获取归约变量的值.**


`prange`返回一个元素为int类型的可迭代对象,这个int对象本质上是迭代任务编号.

在不编译的情况下这个循环会顺序执行,只有通过Cython并带上openmp的参数编译了它才会使用openmp并行化执行.


In [6]:
# 不编译,则顺次执行,就和`range`一样
from cython.parallel import prange

n:int = 30
sum:int  = 0

for i in prange(n, nogil=True):
    sum += i

print(sum)

435


In [7]:
%%cython
# distutils: extra_compile_args=-fopenmp
# distutils: extra_link_args=-fopenmp

# 编译,则并行计算

from cython.parallel import prange

cdef int i
cdef int n = 30
cdef int sum = 0

for i in prange(n, nogil=True):
    sum += i

print(sum)

435


编译版本的prange每次循环相当于一个任务,prange会将每次循环要执行的内容根据`num_threads`,`chunksize`,和`schedule`将整体所有的任务打包为多个`chunk`(可以理解为一个`chunk`就是一批任务)分批分配给一个线程池并行的执行.`num_threads`指定线程池中的线程数,`chunksize`则指定每个`chunk`中的任务数.OpenMP会根据所使用的`schedule`设置分配模式.根据这些模式对应的规则调度任务.这些模式包括:
    
+ `static`静态模式:prange会在进入迭代前先按提供的`chunksize`提前分好任务chunk并直接固定好执行线程;如果没有给出`chunksize`则迭代空间被分成大小相等的块,并且至多一个`chunk`会预先分配给每个线程.适用于调度开销可预测且重要的场景,这种方式可以将问题减少到已知具有大致相同运行时的大小相同的块.
    
+ `dynamic`动态模式:prange会在进入迭代时根据请求分发线程,默认`chunksize`为1,其原理是使用较大数量的较小的块来保持所有线程都在处理.适用于每个块的运行时间不同且无法预测的场景.(默认)
    
+ `guided`有指导模式:与动态调度一样,prange会在进入迭代时根据请求分发线程,但每个`chunk`的大小与未分配迭代数以及参与调度的线程数成正比,也就是说prange刚开始执行时未分配的迭代数大,可用于调度的线程多,所以`chunksize`大;当任务进行到快结束时条件相反所以`chunksize`就小,最小的`chunksize`值如果有设置`chunksize`就是这个设置的值,否则就是`1`
    
+ `runtime`运行时的：调度和`chunksize`取自运行时调度变量,可以通过`openmp.omp_set_schedule()`函数调用或`OMP_SCHEDULE`环境变量进行设置.请注意,这基本上禁用了调度代码本身的任何静态编译时优化.因此可能会显示比在编译时静态配置相同调度策略时更差的性能.
    

`prange`的`nogil`必须为`True`,这相当于循环中的代码块在`with nogil:`上下文管理器中.

`prange`块中的变量我们需要控制其为线程私有,有如下规则:

+ `for i in prange`中的`i`天生私有
+ 块内的`for j in range`中的`j`天生私有
+ 快内的python对象不是私有,并且受GIL限制,需要使用`with gil:`上下文管理器包裹相关操作
+ 块内的变量不会自己设置为私有,我们可以用上面的`parallel`上下文管理器设置他们为私有.

### 典型场景

有以下几种典型场景,他们对应的用法是固定的,可以举一反三结合自己的需要往上套

#### 元素并行操作

最常见的用的最多的就是这种操作,其形式是针对容器中的每个元素执行相同的操作,也就是map-reduce中的map.通常这种操作有如下特征:

+ 输入为一个容器,输出为一个同维度容器
+ 并不是在原始输入上进行修改,而是创建一个新的容器存放结果
+ 不使用全局变量
+ 没有副作用

这种操作多用在数值计算上,一般会结合效率更高的内存视图以及numpy数组一起使用.下面这个例子我们来执行一个将输入的全部元素计算sin值的操作

In [8]:
%%cython
# distutils: extra_compile_args=-fopenmp
# distutils: extra_link_args=-fopenmp


from cython.parallel cimport prange
cimport cython
from libc.math cimport sin

import numpy as np

@cython.boundscheck(False)
@cython.wraparound(False)
def do_sine(double[:,:] input):
    cdef double[:,:] output = np.empty_like(input)
    cdef Py_ssize_t i, j

    for i in prange(input.shape[0], nogil=True):
        for j in range(input.shape[1]):
            output[i, j] = sin(input[i, j])
    return np.asarray(output)



In [9]:
import numpy as np

print(do_sine(np.array([[0.1,0.2,0.3,0.4,0.5],[1.1,1.2,1.3,1.4,1.5]])))

[[0.09983342 0.19866933 0.29552021 0.38941834 0.47942554]
 [0.89120736 0.93203909 0.96355819 0.98544973 0.99749499]]


#### 归并操作

有了map自然有reduce,这种操作一般也有如下特征:

+ 输入是一个容器,输出是一个标量
+ 不使用全局变量
+ 没有副作用

这种操作一样多用在数值计算上,一般会结合效率更高的内存视图以及numpy数组一起使用.下面这个例子我们来执行一个一维数组中全部元素做平方和,最终在开方的操作(也就是l2norm)

In [10]:
%%cython
# distutils: extra_compile_args=-fopenmp
# distutils: extra_link_args=-fopenmp

from cython.parallel cimport prange
cimport cython
from libc.math cimport sqrt

@cython.boundscheck(False)
@cython.wraparound(False)
def l2norm(double[:] x):
    cdef double total = 0
    cdef Py_ssize_t i
    for i in prange(x.shape[0], nogil=True):
        total += x[i]*x[i]
    return sqrt(total)

In [11]:
import array

print(l2norm(array.array("d",[0.1,0.2,0.3,0.4])))

0.5477225575051662


#### 多并行块并行操作

如果有介个并行块要一起执行,但他们之间又有小的串行部分,那么写一个大的并行块会更有效率.这虽然会让串行部分重复计算,但实际上开销反而小.

下面这个例子我们为一维数组进行规范化操作,规范化需要先计算L2范数,然后再对每个元素进行标准化.

In [24]:
%%cython
# distutils: extra_compile_args=-fopenmp
# distutils: extra_link_args=-fopenmp

from cython.parallel cimport parallel, prange
cimport cython
from libc.math cimport sqrt
import numpy as np

@cython.boundscheck(False)
@cython.wraparound(False)
def normalize(double[:] x):
    cdef Py_ssize_t i
    cdef double total = 0
    cdef double norm
    cdef double[:] output = np.empty_like(x)
    with nogil, parallel():
        for i in prange(x.shape[0]):
            total += x[i]*x[i]
        norm = sqrt(total)
        for i in prange(x.shape[0]):
            output[i] = x[i]/norm
    return np.asarray(output)

In [25]:
import numpy as np
print(normalize(np.array([1.1,2.2,3.3,4.4])))

[1. 1. 1. 1.]


这个例子,我们将两个`prange`放在同一个`parallel`上下文中,这会被翻译为如:

```C
#pragma omp parallel private(norm) reduction(+:total)
{
    /* some calculations of array size... */
    #pragma omp for firstprivate(i) lastprivate(i)
    for (__pyx_t_2 = 0; __pyx_t_2 < __pyx_t_3; __pyx_t_2++){
        /* ... */
    }
    norm = sqrt(total);
    #pragma omp for firstprivate(i) lastprivate(i)
    for (__pyx_t_2 = 0; __pyx_t_2 < __pyx_t_3; __pyx_t_2++){
        /* ... */
    }
}
```
这样的形式可以让我们避免跳进跳出omp的`parallel`块,从而减少开支

#### 线程操作结合内存控制

如果我们的计算操作需要用到堆上的内存,那就必须手动控制内存,分配和销毁必须在同一个线程中完成.

下面这个例子我们来在一个二维数组中沿着0轴计算各个列的中位数.

In [31]:
%%cython
# distutils: language = c++
# distutils: extra_compile_args=-fopenmp
# distutils: extra_link_args=-fopenmp

from cython.parallel cimport parallel, prange
from libcpp.vector cimport vector
from libcpp.algorithm cimport nth_element
cimport cython
from cython.operator cimport dereference

import numpy as np

@cython.boundscheck(False)
@cython.wraparound(False)
def median_along_axis0(const double[:,:] x):
    cdef double[::1] out = np.empty(x.shape[1])
    cdef Py_ssize_t i, j

    cdef vector[double] *scratch # 缓存的vector
    cdef vector[double].iterator median_it # 缓存的vector中中间位置迭代器
    with nogil, parallel():
        scratch = new vector[double](x.shape[0]) # 在线程中构造缓存
        try:
            for i in prange(x.shape[1]): # 按列拆分任务
                # 将当列的每一行元素复制到缓存的vector
                for j in range(x.shape[0]):
                    dereference(scratch)[j] = x[j, i]
                median_it = scratch.begin() + scratch.size()//2 # 计算中间位置的index
                nth_element(scratch.begin(), median_it, scratch.end()) #使用`std::nth_element`对中间位置进行部分排序
                # for the sake of a simple example, don't handle even lengths...
                out[i] = dereference(median_it)# 从迭代器中取出中位数
        finally:
            del scratch # 销毁缓存
    return np.asarray(out)

Content of stderr:
ld: warning: dylib (/usr/local/Cellar/gcc/13.1.0/lib/gcc/current/libstdc++.dylib) was built for newer macOS version (11.0) than being linked (10.9)
ld: warning: dylib (/usr/local/Cellar/gcc/13.1.0/lib/gcc/current/libgomp.dylib) was built for newer macOS version (11.0) than being linked (10.9)

In [38]:
x = np.array([
    [0.1,0.2,0.3,0.4,0.5],
    [1.1,1.2,1.3,1.4,1.5],
    [2.1,2.2,2.3,2.4,2.5],
    [1.1,2.2,3.3,0.4,0.5],
])
median_along_axis0(x)

array([1.1, 2.2, 2.3, 1.4, 1.5])

在上面的例子中,我们用prange分配对每一列的计算是一个任务.每一列的计算都用到一个临时的缓存`scratch`用来保存列中出现的数.这个缓存需要在各自的线程中创建和销毁以避免冲突.因此它的创建和销毁过程放在`with parallel():`内让他成为线程私有.
同时为了提高效率,我们并不是每次循环都需要创建和销毁缓存,因为同一线程中缓存是可以复用的,因此缓存的处理在`for i in prange`外.最终它会被翻译为

```C
#pragma omp parallel private(scratch)
{
    scratch = new std::vector<double> ((x.shape[0]))
    #pragma omp for firstprivate(i) lastprivate(i) lastprivate(j) lastprivate(median_it)
    for (__pyx_t_9 = 0; __pyx_t_9 < __pyx_t_10; __pyx_t_9++){
        i = __pyx_t_9;
        /* implementation goes here */
    }
    /* some exception handling detail omitted */
    delete scratch;
}
```

#### 不同线程执行不同的任务

如果手动指定线程数,然后使用`omp.get_thread_num()`识别每个线程.则可以手动在线程之间分配工作.这种用法有点替代python标准库中线程工具的意思,但明显相关工具并不齐全,一旦复杂将难以维护,因此不推荐使用,不过这边还是给个例子

In [26]:
%%cython
# distutils: extra_compile_args=-fopenmp
# distutils: extra_link_args=-fopenmp

from cython.parallel cimport parallel
from openmp cimport omp_get_thread_num

cdef void long_running_task1() nogil:
    with gil:
        print("long_running_task1")

cdef void long_running_task2() nogil:
    with gil:
        print("long_running_task2")

def do_two_tasks():
    cdef int thread_num
    with nogil, parallel(num_threads=2):
        thread_num = omp_get_thread_num()
        if thread_num == 0:
            long_running_task1()
        elif thread_num == 1:
            long_running_task2()

Content of stderr:
ld: warning: dylib (/usr/local/Cellar/gcc/13.1.0/lib/gcc/current/libgomp.dylib) was built for newer macOS version (11.0) than being linked (10.9)

In [27]:
do_two_tasks()

long_running_task2long_running_task1

